In [1]:

import pandas as pd
from pyspark.sql import functions as F
from pyspark.sql.types import *



nist = spark.sql("SELECT * FROM nist_referencespectra5_csv")
#display(nist.select("*"))
nist.show(5)
type(nist)

a = nist.rdd.map(lambda p: p.PeakListClean.split(';')).collect()
b = nist.rdd.map(lambda x: x.X1_1).collect()

mz_arr = []
intensity_arr = []
sample_name = []
mz = 0
intensity = 0


for i in range(0,len(a)-1):
    for item in a[i]:
        mz, intensity = item.split(':')
        #print(mz,",", intensity)
        #if (mz == 0):
        #  bfoundone = True
        mz_arr.append(int(mz))
        intensity_arr.append(int(intensity))
        sample_name.append(b[i])

#if bfoundone == false:
#  mz_arr.append(0)
#  intensity_arr.append(0)
#  sample_name.append("reference")


print(mz_arr[0], intensity_arr[0], sample_name[0])

for i in range(0, max(mz_arr)):
    mz_arr.append(i)
    intensity_arr.append(0)
    sample_name.append("reference")

#import pandas as pd

df = pd.DataFrame({'sample_name': sample_name,'mz':mz_arr, 'intensity':intensity_arr})
print(df.head(1))


#Create PySpark DataFrame Schema
p_schema = StructType([StructField('intensity',IntegerType(),True),StructField('mz',IntegerType(),True),StructField('sample_name',StringType(),True)])


#Create Spark DataFrame from Pandas
df_spark2 = sqlContext.createDataFrame(df, p_schema)

df_spark2.write.saveAsTable("spectra_withtypes")

reshaped_df = df_spark2.groupby('sample_name').pivot('mz').max('intensity').fillna(0)

reshaped_df.show(1)



reshaped_df.write.saveAsTable("spectra_pivotted")




In [2]:
#testing
ref_intensity_parr = sc.parallelize(spark.sql("SELECT * FROM spectra_pivotted").drop("sample_name").rdd.flatMap(lambda x: x).persist())
a_exp = reshaped_df.rdd.map(lambda p: p.PeakList.split(';')).collect()
print(len(a_exp))
print(type(a_exp))